5.4 단순한 계층 구현
    - 사과 쇼핑 예 구현
    - MulLayer: 계산 그래프의 곱셈 노드
    - AddLayer: 계산 그래프의 덧셈 노드
    - 계층 == 신경망의 기능 단위 ex) sigmoid, affine(행렬 내적)

5.4.1 곱셈 계층

In [20]:
class MulLayer:
    def __init__(self):         # 변수 x, y 초기화
        self.x=None
        self.y=None

    def forward(self, x, y):
        self.x=x
        self.y=y
        out=x*y

        return out

    def backward(self, dout):   # 상류에서 넘어온 미분(dout)을 순전파 값을 서로 바꿔 곱한 후 전달
        dx=dout*self.y          # x와 y를 바꿈
        dy=dout*self.x

        return dx, dy

In [21]:
apple=100
apple_num=2
tax=1.1

# 계층들
mul_apple_layer=MulLayer()
mul_tax_layer=MulLayer()

# 순전파
apple_price=mul_apple_layer.forward(apple, apple_num)
price=mul_tax_layer.forward(apple_price, tax)

print(price)

220.00000000000003


In [22]:
# 각 변수에 대한 미분 연산, 호출 순서는 forward와 반대
# backward()의 인수는 '순전파의 출력에 대한 미분'
dprice=1
dapple_price, dtax=mul_tax_layer.backward(dprice)
dapple, dapple_num=mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)

2.2 110.00000000000001 200


5.4.2 덧셈 계층

In [23]:
class AddLayer:
    def __init__(self):         # 초기화 필요 X
        pass

    def forward(self, x, y):    # 인수 x, y를 더해 반환
        out=x+y
        return out

    def backward(self, dout):   # 상류에서 내려온 미분(dout)을 그대로 하류로 전달
        dx=dout*1
        dy=dout*1
        return dx, dy

In [24]:
apple=100
apple_num=2
orange=150
orange_num=3
tax=1.1

# 계층들
mul_apple_layer=MulLayer()
mul_orange_layer=MulLayer()
add_apple_orange_layer=AddLayer()
mul_tax_layer=MulLayer()

# 순전파
apple_price=mul_apple_layer.forward(apple, apple_num)
orange_price=mul_orange_layer.forward(orange, orange_num)
all_price=add_apple_orange_layer.forward(apple_price, orange_price)
price=mul_tax_layer.forward(all_price, tax)

# 역전파
dprice=1
dall_price, dtax=mul_tax_layer.backward(dprice)
dapple_price, dorange_price=add_apple_orange_layer.backward(dall_price)
dorange, dorange_num=mul_orange_layer.backward(dorange_price)
dapple, dapple_num=mul_orange_layer.backward(dapple_price)

print(price)
print(dapple_num, dapple, dorange, dorange_num, dtax)

715.0000000000001
165.0 3.3000000000000003 3.3000000000000003 165.0 650


5.5 활성화 함수 계층 구현
    - 계층 각각을 클래스 하나로 구현

5.5.1 ReLU 계층
    - y=x(x>0),
        0(x<=0)
    - ay/ax=1(x>0), == 0보다 크면 역전파는 상류의 값을 그대로 하류로 전달
            0(x<=0) == 하류로 신호를 전달하지 X

In [25]:
class Relu:
    def __init__(self):
        self.mask=None      # True/False로 구성된 넘파이 배열
                            # 순전파의 입력인 x의 원소 값이 0 이하인 인덱스: True
                            #                              0보다 큰 원소: False
    
    def forward(self, x):
        self.mask=(x<=0)
        out=x.copy()
        out[self.mask]=0

        return out

    def backward(self, dout):
        dout[self.mask]=0
        dx=dout

        return dx

In [19]:
import numpy as np

x=np.array([[1.0, -0.5], [-2.0, 3.0]])
print(x)

mask=(x<=0) # 순전파의 입력 값이 0 이하면 역전파의 값이 0이 되어야 함
print(mask)

[[ 1.  -0.5]
 [-2.   3. ]]
[[False  True]
 [ True False]]
